# Assignment 2

In [ ]:
import os, re, operator, string, math
import pandas as pd
import networkx as nx
import matplotlib.pylab as plt
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from wordcloud import WordCloud, STOPWORDS
from nltk import FreqDist
from fa2 import ForceAtlas2
from collections import Counter

In [ ]:
import pandas as pd
import math
import networkx as nx
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#import preprocessor as p
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk, string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
import os

sns.set(style='white')

# Part 1: Twitter Network Analysis

**_Exercise_ 1: Build the network of retweets.
We will now build a network that has as nodes the Twitter handles of the members of the house, and direct edges between nodes A and B if A has retweeted content posted by B. We will build a weighted network, where the weight of an edge is equal to the number of retweets. You can build the network following the steps below (and you should  be able to reuse many of the functions you have written as part of the exercises during the previous weeks):**

* **Consider the 200 most recent tweets written by each member of the house (use the files [here](https://github.com/suneman/socialgraphs2019/tree/master/files/data_twitter/tweets_2019.zip/). For each file, use a regular expression to find retweets and to extract the Twitter handle of the user whose content was retweeted. All retweets begin with "*RT @originalAuthor:*", where "*originalAuthor*" is the handle of the user whose content was retweeted (and the part of the text you want to extract).**

The [file](https://github.com/suneman/socialgraphs2019/tree/master/files/data_twitter/tweets_2019.zip/) has been downloaded and unzipped in the current path.

In [ ]:
# Initializing variables
all_rts = {}
folder = "tweets_2019_full/"

# Read each file
for filename in os.listdir(path=folder):
    all_rts[filename] = []
    filepath = "{}{}".format(folder, filename)
    f = open(filepath,"r+",encoding='utf-8')
    lines = f.readlines()
    f.close()
    # Find all retweets within the current file
    for line in lines:
        retweets = re.findall(r"RT @\w+", line)
        counter = 0
        # Save the first 200 retweets
        for retweet in retweets:
            if counter<200:
                all_rts[filename].append(retweet[4:])
                counter += 1

In [ ]:
folderPath = "tweets_2019_full/"
tweets = {}

for file in sorted(os.listdir(folderPath)):
    filepath = os.path.join(folderPath, file)
    f = open(filepath, 'r', encoding="utf8")
    tweets[file] = f.read()
    f.close()

* **For each retweet, check if the handle retweeted is, in fact, the handle of a member of the house. If yes, keep it. If no, discard it.**

Also the [file](https://github.com/suneman/socialgraphs2019/blob/master/files/data_twitter/H115_tw_2019.csv) containing the data of the 115th Congress has been downloaded in the current path.

In [ ]:
# Load the data of all the politicians
df = pd.read_csv('H115_tw_2019.csv')
twitterHandle = df
df = df.set_index('tw_name') # setting 'tw_name' column as index

In [ ]:
tw_names = []
for i in range(len(twitterHandle)):
    
    #name = twitterHandle[i]['tw_name']
    tw_names.append(twitterHandle.loc[i]['tw_name'])
    

##Filtering out duplicates
tw_names =  set(tw_names)

In [ ]:
# Initializing variable
pol_rts = {}

# Filter retweets
for key, values in all_rts.items():
    pol_rts[key] = []
    for value in values:
        # Save retweet only if belongs to members of the house
        if value in df.index:
            real_name = df.loc[value]['WikiPageName']
            if key != real_name: # avoiding self-loops
                pol_rts[key].append(real_name)

* **Use a NetworkX [`DiGraph`](https://networkx.github.io/documentation/development/reference/classes.digraph.html) to store the network. Use weighted edges to account for multiple retweets. Also store the party of each member as a node attribute (use the data in [this file](https://github.com/suneman/socialgraphs2019/blob/master/files/data_twitter/H115_tw_2019.csv). Remove self-loops (edges that connect a node with itself).**

In [ ]:
df['tw_name'] = df.index
df = df.set_index('WikiPageName') # setting 'WikiPageName' column as index

In [ ]:
# Create the graph
G = nx.DiGraph()

# Add nodes
for index, row in df.iterrows():
    G.add_node(index, party=row[0])
    
# Add edges
for pol, links in pol_rts.items():
    links_counter = Counter(links)
    for link, count in links_counter.items():
        G.add_edge(pol, link, weight=count)

 **_Exercise_ 2: Visualize the network of retweets and investigate differences between the parties.**
 * **Visualize the network using the [Networkx draw function](https://networkx.github.io/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw.html#networkx.drawing.nx_pylab.draw), and nodes coordinates from the force atlas algorithm. *Hint: use an undirected version of the graph to find the nodes positions for better results, but stick to the directed version for all measurements.* Plot nodes in colors according to their party (e.g. 'red' for republicans and 'blue' for democrats) and set the node-size to be proportional to total degree.**

In [ ]:
# Create 'forceatlas2' configuration
forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=False,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.5,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=0.0001,
                        strongGravityMode=False,
                        gravity=0.5,

                        # Log
                        verbose=True)

# Create undirected graph
Gu = G.to_undirected()

# Calculate positions
positions = forceatlas2.forceatlas2_networkx_layout(Gu, pos=None, iterations=2000)

In [ ]:
# Map parties to colors
parties_map = {'Republican':'Red',
              'Democratic':'Blue'}

# Plot graph
plt.figure(figsize=[15,15])
nx.draw_networkx_nodes(Gu, positions, with_labels=False, alpha=0.7, 
                       node_size=[5*s for s in dict(Gu.degree(Gu.nodes())).values()],
                       node_color=[c for c in [parties_map.get(Gu.node[n]['party']) for n in Gu.nodes()]])
nx.draw_networkx_edges(Gu, positions, edge_color="black", alpha=0.2, arrows=False)
plt.axis('off')
plt.show()

  * **Compare the network of Retweets with the network of Wikipedia pages (generated during Week 5). Do you observe any differences? How do you explain them?**

Yes, there are some differences.

The Wikipedia-graph has interactions between nodes of different parties, resulting in a very mixed graph, but the Retweet-graph has only few interactions between members of different parties. As a result, the Retweet-graph has two different zones, one per party, with 2-3 isolated members of the other party. Both zones are connected by some edges just in the center of the graph.

* **Now set the node-size tob proportional to betweenness centrality. Do you observe any changes?**

In [ ]:
# Calculate betweenness centrality
btwss = nx.betweenness_centrality(G)

# Plot graph
plt.figure(figsize=[15,15])
nx.draw_networkx_nodes(Gu, positions, with_labels=False, alpha=0.7, 
                       node_size=[10000*btwss[n] for n in Gu.nodes()],
                       node_color=[c for c in [parties_map.get(Gu.node[n]['party']) for n in Gu.nodes()]])
nx.draw_networkx_edges(Gu, positions, edge_color="black", alpha=0.2, arrows=False)
plt.axis('off')
plt.show()

The members that have links with members of the other party increase considerably in size compared to those that do not. The size of a node is proportional to the number of shortest paths between all node pairs that run along that node.

* **Repeat the point above using eigenvector centrality. Again, do you see a difference? Can you explain why based on what eigenvector centrality measures?**

In [ ]:
# Calculate eigenvector centrality
eigen = nx.eigenvector_centrality(G)

# Plot graph
plt.figure(figsize=[15,15])
nx.draw_networkx_nodes(Gu, positions, with_labels=False, alpha=0.7, 
                       node_size=[2000*eigen[n] for n in Gu.nodes()],
                       node_color=[c for c in [parties_map.get(Gu.node[n]['party']) for n in Gu.nodes()]])
nx.draw_networkx_edges(Gu, positions, edge_color="black", alpha=0.2, arrows=False)
plt.axis('off')
plt.show()

The nodes located in the republican side of the graph are bigger than the ones in the democratic side.

Using the eigenvector centrality the nodes with connections to high-scoring nodes will have a higher score. It's easy to check that the republicans have more interactions between them taking a look to the list of nodes with a higher degree (check the next section of this exercise). As only a few links connect both zones, the average score in the republican zone remains high.

* **Who are the three nodes with highest degree within each party? And what is their eigenvector centrality? And their betweenness centrality?**

In [ ]:
# Calculate the degree of each node and order the results
degree = dict(G.degree(G.nodes()))
degree_rank = dict(sorted(degree.items(), key=operator.itemgetter(1), reverse=True))

# Print values for top3 nodes of each party
print("The 3 politicians of each party with higher degree are:")
dem_count = 0
rep_count = 0
for key, value in degree_rank.items():
    if G.node[key]['party']=='Democratic' and dem_count<3:
        print("The democrat "+df.loc[key]['Name']+" with degree "+str(value)
              +", eigenvector centrality "+str(round(eigen[key],ndigits=3))
              +" and betweenness centrality "+str(round(btwss[key],ndigits=3)))
        dem_count += 1
    elif G.node[key]['party']=='Republican' and rep_count<3:
        print("The republican "+df.loc[key]['Name']+" with degree "+str(value)
              +", eigenvector centrality "+str(round(eigen[key],ndigits=3))
              +" and betweenness centrality "+str(round(btwss[key],ndigits=3)))
        rep_count += 1
    elif dem_count==3 and rep_count==3:
        break

* **Plot (on the same figure) the distribution of outgoing strength for the republican and democratic nodes repectively (i.e. the sum of the weight on outgoing links). Which party is more active in retweeting other members of the house?**

In [ ]:
# Initializing variables
N = len(list(G.nodes))
dem_out_w = []
rep_out_w = []

# Calculating the outgoing strength
for node in G.nodes:
    if G.node[node]['party']=='Democratic':
        dem_out_w.append(sum(G[u][v]['weight'] for (u,v) in G.out_edges([node])))
    else:
        rep_out_w.append(sum(G[u][v]['weight'] for (u,v) in G.out_edges([node])))

In [ ]:
# Plot distributions
fig=plt.figure(figsize=[15,5])
fig.add_subplot(1,2,1)
plt.hist(dem_out_w, bins=N//10)
plt.title('Democratic outgoing strength')
plt.xlabel('sum(out_edges_weight)')
plt.ylabel('N(sum(out_edges_weight))')
plt.xlim(-3,105)
plt.ylim(0,70)
fig.add_subplot(1,2,2)
plt.hist(rep_out_w, bins=N//10, color='Red')
plt.title('Republican outgoing strength')
plt.xlabel('sum(out_edges_weight)')
plt.ylabel('N(sum(out_edges_weight))')
plt.xlim(-3,105)
plt.ylim(0,70)
plt.show()

The members of the Republican party make more retweets than the democrats.

* **Find the 3 members of the republican party that have retweeted tweets from democratic members most often. Repeat the measure for the democratic members. Can you explain your results by looking at the Wikipedia pages of these members of the house?**

In [ ]:
# Initializing variables
dem_in_d = {}
rep_in_d = {}

# Calcule most retweeted politicians
for node in G.nodes:
    counter = 0
    if G.node[node]['party']=='Democratic':
        for i in G.in_edges([node]):
            if G.node[i[0]]['party']=='Republican':
                counter += 1
        dem_in_d[node] = counter
    else:
        for i in G.in_edges([node]):
            if G.node[i[0]]['party']=='Democratic':
                counter += 1
        rep_in_d[node] = counter

In [ ]:
# Order the results
dem_in_d_rank = dict(sorted(dem_in_d.items(), key=operator.itemgetter(1), reverse=True)[:3])
rep_in_d_rank = dict(sorted(rep_in_d.items(), key=operator.itemgetter(1), reverse=True)[:3])

# Print the results
print("The 3 most retweeted democratic politicians by the republicans are:")
for key, value in dem_in_d_rank.items():
    print(df.loc[key]['Name']+" with "+str(value)+" retweets")
print("\nThe 3 most retweeted republican politicians by the democrats are:")
for key, value in rep_in_d_rank.items():
    print(df.loc[key]['Name']+" with "+str(value)+" retweets")

On one hand, some facts that could explain why some republicans retweeted these democrats are:

- Gene Green, despite the Democratic leadership's general disapproval of the Iraq war, voted against measures aimed at placing a timetable on military withdrawal.
- Ted Deutch spoke out in favor of expanded gun control legislation after the Stoneman Douglas High School shooting (where 17 passed away).
- Business groups have praised Derek Kilmer for being one of the most pro-business Democrats in Olympia.

On the other hand, some facts that could explain why some democrats retweeted these republicans are:

- Justin Amash has described himself as a libertarian, dissenting from both Republican and Democratic leaderships more frequently than the vast majority of Republican members of Congress. On July 4, 2019, he announced that he was leaving the Republican Party to become an independent.
- Adam Kinzinger was ranked as the 40th most bipartisan (by measuring the frequency each member's bills attract co-sponsors from the opposite party and each member's co-sponsorship of bills by members of the opposite party) member of the U.S. House of Representatives during the 114th United States Congress.
- David Schweikert supports allowing veterans access to medical marijuana, if legal in their state, per their Veterans Health Administration doctor's recommendation and voted twice in support of this in the Veterans Equal Access Amendment.

# Exercise 3: Community detection.

* **Use your favorite method of community detection to find communities in the full house of representatives network. Report the value of modularity found by the algorithm. Is it higher or lower than what you found for the Wikipedia network (Week 7)? Comment on your result.**

We are using the louvain community detection algorithm to detect communities in the full house representatives network.

In [ ]:
import community

partition = community.best_partition(Gu)
#calculate the number of communities
nb_communities = len(set(partition.values()))

# add a new "community attribute" to the nodes of the undirected graph
nx.set_node_attributes(Gu, name='community', values=partition)

In [ ]:
print(f"Calculated number of communities: {nb_communities}")

In [ ]:
# create a dictionary to store the members of each community as a list
communities = {}
for group in set(partition.values()): #<-- partition is a dictionary with names 
                                      #      as keys and communities as values
    members = [key for key, value in partition.items() if value==group]
    communities[group] = members

In [ ]:
party_partition = df.reset_index()[['WikiPageName', 'Party']].copy()
# map party to 1 or zero repsectively
party_partition.Party = party_partition.Party.apply(lambda x: 0 if x=='Republican' else 1)
# create a dictionary same as community dictionary
party_partition = party_partition.set_index("WikiPageName").to_dict()['Party']

In [ ]:
print(f"Modularity of Louvain Partition: {community.modularity(partition, Gu)}")

In [ ]:
print(f"Modularity Based on Parties: {community.modularity(party_partition, Gu)}")

The modularity found by the algorithm is 0.485 which is substantially larger than the one computed at week 7 ( around 0.15 ),  as well as the modularity based on party membership.

This tells us that this partitioning describes better the network structure of the house of representatives. 
Also we can say that the data mined from twitter gave us a lot more information about the network structure as well as about the conections between person and between parties, than the wikipedia data.

* **Visualize the network, using the Force Atlas algorithm. This time assign each node a different color based on their *community*. Describe the structure you observe.**

In [ ]:
# set our color palette
palette = sns.color_palette("muted", n_colors=nb_communities)

# select color based on community
node_color  = []
for node in Gu.nodes(data=True):
    node_color.append(palette[int(node[1]['community'])])

In [ ]:
plt.figure(figsize=(20,20))
nx.draw_networkx(Gu, positions, node_size=[100*v  for v in dict(nx.degree(Gu)).values()], with_labels=False, width=0.2, node_color=node_color, alpha=0.7)
plt.axis('Off')
plt.show()

* **Compare the communities found by your algorithm with the parties by creating a matrix $\mathbf{D}$ with dimension $(B \times C$, where $B$ is the number of parties and $C$ is the number of communities. We set entry $D(i,j)$ to be the number of nodes that party $i$ has in common with community $j$. The matrix $\mathbf{D}$ is what we call a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix).**


In [ ]:
confusion_matrix = np.zeros((2, nb_communities))
for node in Gu.nodes(data=True):
    confusion_matrix[party_partition[node[0]], node[1]['community']] += 1
    
confusion_matrix = pd.DataFrame(confusion_matrix, 
                                index=['Republican', 'Democratic'],
                                columns=range(nb_communities))

In [ ]:
confusion_matrix

We will also normalise the confusion matrix

In [ ]:
confusion_matrix = confusion_matrix/len(list(Gu.nodes))

In [ ]:
#plot confusion matrix
sns.heatmap(confusion_matrix)

* **Use the confusion matrix to explain how well the communities you've detected correspond to the parties. Consider the following questions**

#### Observations

- group 0 has the bulk of the democrats and group 2 most of the republicans
- republicans are in general spread into more groups
- there are many (4) groups that republicans and democrats coexist in the same numbers.

##### Does the community detection algorithm sub-divide the parties? Do you know anything about American politics that could explain such sub-divisions? Answer in your own words.

The community detection algorithm seems to detect sub-divisions of the parties. This could be senators from same or nearby states or for specific groups inside parties we really well defined jobs e.g ethics committee. Another group detected by the algorithm is that of the independant senators.

# Part 2: What do republican and democratic members tweet about?

**_Exercise_ 4: TF-IDF of the republican and democratic tweets.**

**We will create two documents, one containing the words extracted from tweets of republican members, and the other for Democratic members. We will then use TF-IDF to compare the content of these two documents and create a word-cloud. The procedure you should use is exactly the same you used in exercise 2 of week 7. The main steps are summarized below:**

* **Create two large documents, one for the democratic and one for the republican party. Tokenize the pages, and combine the tokens into one long list including all the pages of the members of the same party.**
  * **Exclude the twitter handles of other members.**
  * **Exclude punctuation.**
  * **Exclude stop words (if you don't know what stop words are, go back and read NLPP1e again).**
  * **Exclude numbers (since they're difficult to interpret in the word cloud).**
  * **Set everything to lower case.**
  * **Compute the TF-IDF for each document.**

In [ ]:
# Initializing variables
folder = "tweets_2019_full/"
stop_words = set(stopwords.words('english'))
stop_words.add('amp') # in order to avoid '&amp' symbols

# Create files
dem_filepath = "democrats.txt"
rep_filepath = "republicans.txt"
dem_file = open(dem_filepath,"w+",encoding='utf-8')
rep_file = open(rep_filepath,"w+",encoding='utf-8')
dem_file.close()
rep_file.close()

In [ ]:
# Function to clean & tokenize the words within a tweet
def clean_tweet(tweet):

    clean_tweet = re.sub(r'@\w+', '', tweet) # Remove mentions
    clean_tweet = re.sub(r'#\w+', '', clean_tweet) # Remove hashtags
    clean_tweet = re.sub(r'http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
                         '', clean_tweet) # Remove URL's
    clean_tweet = re.sub(r'\d+(.\d+)?', '', clean_tweet) # Remove numbers
    clean_tweet = clean_tweet.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation symbols
    clean_tweet = re.sub(r'…', '', clean_tweet) # Remove consecutive dots (special character)
    clean_tweet = re.sub(r'[\“\”\’]', '', clean_tweet) # Remove special quotes and double quotes
    clean_tweet = re.sub(r' +', ' ', clean_tweet) # Remove unnecesary whitespaces
    clean_tweet = ' '.join( [w for w in clean_tweet.split() if len(w)>1] ) # Remove isolated letters
    tweet_tokens = word_tokenize(clean_tweet.lower())
    filtered_tokens = []
    for w in tweet_tokens: 
        if w not in stop_words: # Filter stop words
            filtered_tokens.append(w) 
    clean_tweet = ' '.join( [w for w in filtered_tokens] )
    
    return clean_tweet

In [ ]:
import os 

df = pd.read_csv("H115_tw_2019.csv")
df = df.set_index("WikiPageName")

# Read the files of the members of the house
for filename in os.listdir(path=folder):
    filepath = "{}{}".format(folder, filename)
    f_read = open(filepath,"r+",encoding='utf-8')
    lines = f_read.readlines()
    f_read.close()
    current_filepath = ''
    # Check the member's party
    if df.loc[filename]['Party']=='Democratic':
        current_filepath = dem_filepath
    else:
        current_filepath = rep_filepath
    f = open(current_filepath,"a+",encoding='utf-8')
    
    # Clean the tweets
    for line in lines:
        retweet = re.findall(r"RT @\w+", line) # Discard retweets
        if not retweet:
            filtered_tweet = clean_tweet(line)
            f.write(filtered_tweet)        
    f.close()

In [ ]:
# Read files
dem_file = open(dem_filepath,"r+",encoding='utf-8')
rep_file = open(rep_filepath,"r+",encoding='utf-8')
dem_words = dem_file.read()
rep_words = rep_file.read()
dem_file.close()
rep_file.close()

In [ ]:
# Calculate TF-IDF
def calc_tf_idf(tokens,doc_token_sets):
    
    fdist = nltk.FreqDist(tokens)
    ndocs = len(doc_token_sets)
    tf_idf_list = []
    for item in fdist.most_common():
        word = item[0]
        cnt = 0
        for i in range(ndocs):
            if word in doc_token_sets[i]:
                cnt += 1  
        try:
            idf = math.log(ndocs/cnt)
            tf = item[1]

            tf_idf = tf*idf
            tf_idf_list.append((word,tf_idf))
        except:
            print('Exception')
    tf_idf_list = sorted(tf_idf_list, key=lambda x: x[1],reverse = True)  
    
    return tf_idf_list

In [ ]:
from nltk import FreqDist

# Tokenize
dem_tokens = word_tokenize(dem_words)
rep_tokens = word_tokenize(rep_words)

<font color='red'>Please be pacient, the next cell takes a long time to compute...</font>

In [ ]:
# Compute TF-IDF
dem_tfidf = calc_tf_idf(dem_tokens, list(set(dem_tokens)))
rep_tfidf = calc_tf_idf(rep_tokens, list(set(rep_tokens)))

In [ ]:
# Order words by TF-IDF
sorted_dem_tfidf = sorted(dem_tfidf, key=lambda kv: kv[1], reverse=True)
sorted_rep_tfidf = sorted(rep_tfidf, key=lambda kv: kv[1], reverse=True)

In [ ]:
# Print top10 results for each party
print("Democrats top10 words:")
print(str(sorted_dem_tfidf[:10])+"\n")
print("Republicans top10 words:")
print(sorted_rep_tfidf[:10])

* **Now, create word-cloud for each party. Are these topics less "boring" than the Wikipedia topics from Week 7? Why?  Comment on the results.**

In [ ]:
def blue_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl(230,100%%, %d%%)" % np.random.randint(49,51))
def red_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl(355,100%%, %d%%)" % np.random.randint(49,51))

dem_wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color='white',
                     ).generate(dem_words)
dem_wordcloud.recolor(color_func = blue_color_func)
rep_wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color='white'
                     ).generate(rep_words)
rep_wordcloud.recolor(color_func = red_color_func)

fig=plt.figure(figsize=[15,10])
fig.add_subplot(1,2,1)
plt.imshow(dem_wordcloud)
plt.axis('off')
fig.add_subplot(1,2,2)
plt.imshow(rep_wordcloud)
plt.axis('off')
plt.show()

These topics are more interesting because the nature of the text. The tweets are limited to 280 characters, so the politicians have to express their ideas and slogans briefly to fit them. In the case of the Wikipedia pages the texts are not written by politicians and contains some general information about each member of the house, so the word-cloud contains less relevant words to characterize each party.

# Part 3 - Sentiment analysis

## Exercise 5: Sentiment over the Twitter data.

- **Download the LabMT wordlist. It's available as supplementary material from Temporal Patterns of Happiness and Information in a Global Social Network: Hedonometrics and Twitter (Data Set S1). Describe briefly how the list was generated.**

The LabMT wordlist contains a happiness ratings of over 10.000 english words. The wordlist is downloaded from the following link: [Data](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0026752#pone.0026752.s001). The list was generated using the Amazon's Mechanical Turk Service. 

In [ ]:
filePath = 'LabMT_wordlist.txt'

labmt = pd.read_csv(filePath, sep='\t')

- Creating a dictionary to store the words from the lamMT for a easy look-up

In [ ]:
word_happiness={}
for word in labmt['word']:
    word_idx = labmt[labmt['word']==word].index[0]
    word_happiness[word] = labmt.loc[word_idx,'happiness_average']

* **Based on the LabMT word list, write a function that calculates sentiment given a list of tokens (the tokens should be lower case, etc).**

In [ ]:
def calc_sentiment(tokens,word_happiness):
    
    happiness_avg = [word_happiness[t.lower()] for t in tokens if t.lower() in word_happiness.keys()]
    
    # tweets which do not contain any words in the happiness list return return 0
    if len(happiness_avg)>0:
        return sum(happiness_avg)/len(happiness_avg)
    else:
        return 0

    return happiness_avg.mean()

* **Create two lists: one containing tweets by democratic members, and the other with the tweets of republican members.**

In [ ]:
democratic_tweets = []
republican_tweets = []

for key in tw_names:
    
    wikiName = twitterHandle.loc[twitterHandle['tw_name'] == key]['WikiPageName'].values[0]
    
    try:
        
        tweet_text = tweets[wikiName]
        tweet_list = tweet_text.split('\n')
    
    except:
        
        print(wikiName)
    
    
    party = twitterHandle.loc[twitterHandle['tw_name'] == key]['Party'].values[0]

    if party == 'Democratic':
        democratic_tweets += tweet_list
    elif party == 'Republican':
        republican_tweets += tweet_list

The tweets file seems to be missing few records. 

* **Calculate the sentiment of each tweet and plot the distribution of sentiment for each of the two lists. Are there significant differences between the two? Which party post more positive tweets?**

- Calculating the sentiments of tweets made by democratic politicians

In [ ]:
## Initializing the variable to store sentiments of democrats
democratic_tweet_sentiments = []

#cnt = 0
democratic_tweet_sentiments = np.zeros(len(democratic_tweets))
for i in range(len(democratic_tweets)):
    #if cnt%1000==0:
        #print(cnt)
    token = democratic_tweets[i]
    tweet_tokens = nltk.word_tokenize(token.lower())
    democratic_tweet_sentiments[i] = calc_sentiment(tweet_tokens,word_happiness)
    #cnt += 1

- Calculating the sentiments of tweets made by republican

In [ ]:
## Initializing the variable to store sentiments of republican
republican_tweet_sentiments = []

#cnt = 0
republican_tweet_sentiments = np.zeros(len(republican_tweets))
for i in range(len(republican_tweets)):
    #if cnt%1000==0:
        #print(cnt)
    token = republican_tweets[i]
    tweet_tokens = nltk.word_tokenize(token.lower())
    republican_tweet_sentiments[i] = calc_sentiment(tweet_tokens,word_happiness)
    #cnt += 1

- Filtering out the sentiments whose happiness score is 0 or less than zero

In [ ]:
filtered_idx = np.argwhere(democratic_tweet_sentiments>0)
filtered_dem_sent = democratic_tweet_sentiments[filtered_idx]
filtered_dem_tweets = []

for idx in filtered_idx:
    
    filtered_dem_tweets.append(democratic_tweets[int(idx)])

In [ ]:
filtered_idx = np.argwhere(republican_tweet_sentiments>0)
filtered_rep_sent = republican_tweet_sentiments[filtered_idx]
filtered_rep_tweets = []

for idx in filtered_idx:
    
    filtered_rep_tweets.append(republican_tweets[int(idx)])

- Visualizing the sentiments of both parties using histogram

In [ ]:
histbins = np.linspace(min(min(filtered_rep_sent),min(filtered_dem_sent)),max(max(filtered_rep_sent),max(filtered_dem_sent)),30)

histbins1 = [item for sublist in histbins for item in sublist]

plt.figure(figsize=(8,6))
plt.hist(filtered_dem_sent,color='b',bins=histbins1,alpha=0.5)
plt.hist(filtered_rep_sent,color='r',bins=histbins1,alpha=0.5)
plt.xlabel('Sentiment score')
plt.ylabel('Count')
plt.legend(['Democratic','Republican'])
plt.xlim(3,8)
plt.show()

The democratic politicians seems to be tweeting slightly more positively compared to their political opponent the republican party. 

* **Compute the average $m$ and standard deviation $\sigma$ of the Tweets sentiment (considering tweets by both republican and democrats).**

In [ ]:
overall_sent = np.concatenate((filtered_dem_sent,filtered_rep_sent))
overall_sent_mean = np.mean(overall_sent)
overall_sent_std = np.std(overall_sent)

print('The mean of all the sentiments of the tweets made by both parties is {}'.format(overall_sent_mean))
print('The standard deviation of all the sentiments of the tweets made by both parties is {}'.format(overall_sent_std))

* **Defining a function to compute tf_idf**

In [ ]:
def calc_tf_idf(tokens,doc_token_sets):
    # calculate tf-idf
    fdist = nltk.FreqDist(tokens)
    ndocs = len(doc_token_sets)
    tf_idf_list = []
    for item in fdist.most_common():
        word = item[0]
        cnt = 0
        for i in range(ndocs):
            if word in doc_token_sets[i]:
                cnt += 1
                
        try:
            
            idf = math.log(ndocs/cnt)
            tf = item[1]

            tf_idf = tf*idf
            tf_idf_list.append((word,tf_idf))
        
        except:
            
            print('Exception')
            
    tf_idf_list = sorted(tf_idf_list, key=lambda x: x[1],reverse = True)  
    
    return tf_idf_list

In [ ]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english') + [u'rt'] # add retweet tag to stopwords

In [ ]:
dem_tokens = []
dem_doc_tokens = []
rep_tokens = []
rep_doc_tokens = []
for key in tw_names:
    
    wikiName = twitterHandle.loc[twitterHandle['tw_name'] == key]['WikiPageName'].values[0]
    
    try:
        
        text = tweets[wikiName]#.decode('unicode-escape')
        tokens = nltk.word_tokenize(text)

        # set to lower case
        tokens = [t.lower() for t in tokens]
        # check if word is alphabetic (removes numbers, hashtags, @ and punctuation) and remove stopwords
        tokens = [t for t in tokens if t not in stopwords and t.isalpha()]
        # remove twitter handles
        tokens = [t for t in tokens if t not in tw_names]

        party = twitterHandle.loc[twitterHandle['tw_name'] == key]['Party'].values[0]

        if party == 'Democratic':
            dem_tokens += tokens
            dem_doc_tokens.append(set(tokens))
        elif party == 'Republican':
            rep_tokens += tokens
            rep_doc_tokens.append(set(tokens))
    
    except:
        print(wikiName)

It seems like the twitter files for couple of politician is missing in the twitter dataset

* **Now only tweets with sentiment lower than $m-2\sigma$. We will refer to them as negative tweets. Build a list containing negative tweets written by democrats, and one for republicans.**

- Creating a list of negative tweets made by democrats

In [ ]:
## Filtering out all the democratic tweets with sentiment ratings over m - 2 std
neg_idx = np.argwhere(filtered_dem_sent<(overall_sent_mean-2*overall_sent_std))[:,0]
neg_dem_tweets = []
for idx in neg_idx:
    neg_dem_tweets.append(filtered_dem_tweets[int(idx)])

- Extracting all negative tweets made by democrats

In [ ]:
neg_dem_tokens = []
for tw in neg_dem_tweets:
    tokens = nltk.word_tokenize(tw)
    # set to lower case
    tokens = [t.lower() for t in tokens]
    # check if word is alphabetic (removes numbers, hashtags, @ and punctuation) and remove stopwords
    tokens = [t for t in tokens if t not in stopwords and t.isalpha()]
    # remove twitter handles
    tokens = [t for t in tokens if t not in tw_names]
    
    neg_dem_tokens += tokens

 - Computing tf-idf for democratic negative sentiments

In [ ]:
tf_idf_neg_dem = calc_tf_idf(neg_dem_tokens,dem_doc_tokens)

In [ ]:
neg_dem_str = ''
for item in tf_idf_neg_dem:
    word = item[0]
    count = int(item[1])
    neg_dem_str += (word + ' ')*count

- Negative democratic wordcloud

In [ ]:
wc_neg_dem = WordCloud(background_color="white",width=400, height=400,colormap=plt.cm.Blues,collocations=False)
wc_neg_dem.generate(neg_dem_str)
plt.figure(figsize=(10,10))
plt.imshow(wc_neg_dem,interpolation='bilinear')
plt.axis("off")
plt.show()

- Creating a list of negative tweets made by republican

In [ ]:
## Filtering out all the republican tweets with sentiment ratings over m - 2 std
neg_idx = np.argwhere(filtered_rep_sent<(overall_sent_mean-2*overall_sent_std))[:,0]
neg_rep_tweets = []
for idx in neg_idx:
    neg_rep_tweets.append(filtered_rep_tweets[int(idx)])

In [ ]:
neg_rep_tokens = []
for tw in neg_rep_tweets:
    tokens = nltk.word_tokenize(tw)
    # set to lower case
    tokens = [t.lower() for t in tokens]
    # check if word is alphabetic (removes numbers, hashtags, @ and punctuation) and remove stopwords
    tokens = [t for t in tokens if t not in stopwords and t.isalpha()]
    # remove twitter handles
    tokens = [t for t in tokens if t not in tw_names]
    
    neg_rep_tokens += tokens

 - Computing tf-idf for republican negative sentiments

In [ ]:
tf_idf_neg_rep = calc_tf_idf(neg_rep_tokens,rep_doc_tokens)

In [ ]:
neg_rep_str = ''
for item in tf_idf_neg_rep:
    word = item[0]
    count = int(item[1])
    neg_rep_str += (word + ' ')*count

- Negative republican wordcloud

In [ ]:
wc_neg_rep = WordCloud(background_color="white",width=400, height=400,colormap=plt.cm.Reds,collocations=False)
wc_neg_rep.generate(neg_rep_str)
plt.figure(figsize=(10,10))
plt.imshow(wc_neg_rep,interpolation='bilinear')
plt.axis("off")
plt.show()

**Conclusion of negative sentiments:**  

The democrats seems to be focusing on topics such as violence, death and imprisonment. However, the republicans seems to be focusing on other topics such as weather, economy, guilt and failure.

* **Repeat the point above, but considering positive tweets (e.g. with sentiment larger than $m+2\sigma$). Comment on your results.**

- Creating a list of positive tweets made by democrats

In [ ]:
## Filtering out all the tweets of democrats with sentimentaL score less than m + 2 std
pos_idx = np.argwhere(filtered_dem_sent>(overall_sent_mean+2*overall_sent_std))[:,0]
pos_dem_tweets = []
for idx in pos_idx:
    pos_dem_tweets.append(filtered_dem_tweets[int(idx)])

In [ ]:
pos_dem_tokens = []
for tw in pos_dem_tweets:
    tokens = nltk.word_tokenize(tw)
    # set to lower case
    tokens = [t.lower() for t in tokens]
    # check if word is alphabetic (removes numbers, hashtags, @ and punctuation) and remove stopwords
    tokens = [t for t in tokens if t not in stopwords and t.isalpha()]
    # remove twitter handles
    tokens = [t for t in tokens if t not in tw_names]
    
    pos_dem_tokens += tokens

 - Computing tf-idf for democrats positive sentiments

In [ ]:
tf_idf_pos_dem = calc_tf_idf(pos_dem_tokens,dem_doc_tokens)

In [ ]:
pos_dem_str = ''
for item in tf_idf_pos_dem:
    word = item[0]
    count = int(item[1])
    pos_dem_str += (word + ' ')*count

- Positive democratic wordcloud

In [ ]:
wc_pos_dem = WordCloud(background_color="white",width=400, height=400,colormap=plt.cm.Blues,collocations=False)
wc_pos_dem.generate(pos_dem_str)
plt.figure(figsize=(10,10))
plt.imshow(wc_pos_dem,interpolation='bilinear')
plt.axis("off")
plt.show()

- Creating a list of positive tweets made by republican

In [ ]:
pos_idx = np.argwhere(filtered_rep_sent>(overall_sent_mean+2*overall_sent_std))[:,0]
pos_rep_tweets = []
for idx in pos_idx:
    pos_rep_tweets.append(filtered_rep_tweets[int(idx)])

In [ ]:
pos_rep_tokens = []
for tw in pos_rep_tweets:
    tokens = nltk.word_tokenize(tw)
    # set to lower case
    tokens = [t.lower() for t in tokens]
    # check if word is alphabetic (removes numbers, hashtags, @ and punctuation) and remove stopwords
    tokens = [t for t in tokens if t not in stopwords and t.isalpha()]
    # remove twitter handles
    tokens = [t for t in tokens if t not in tw_names]
    
    pos_rep_tokens += tokens

 - Computing tf-idf for republican positive sentiments

In [ ]:
tf_idf_pos_rep = calc_tf_idf(pos_rep_tokens,rep_doc_tokens)

In [ ]:
pos_rep_str = ''
for item in tf_idf_pos_rep:
    word = item[0]
    count = int(item[1])
    pos_rep_str += (word + ' ')*count

- Positive republican wordcloud

In [ ]:
wc_pos_rep = WordCloud(background_color="white",width=400, height=400,colormap=plt.cm.Reds,collocations=False)
wc_pos_rep.generate(pos_rep_str)
plt.figure(figsize=(10,10))
plt.imshow(wc_pos_rep,interpolation='bilinear')
plt.axis("off")
plt.show()

**Conclusion of Positive sentiments:**  

Both political parties seems to be focusing on greetings and annual holidays and independence.